In [1]:
from openai import OpenAI
import json

First, let's initialize the client:

In [2]:
client = OpenAI()

Here is the list of text messages we want to process:

In [3]:
text_messages = [
    "The service here is very good!",
    "The service here is good.",
    "The service here is ok.",
    "The service here is not very good.",
    "The service here is terrible!",
]

Now let's create the batch file.
We have some reuseable code for prompts from before.

In [4]:
system_prompt = "You are an expert on sentiment analysis. Your job is to evaluate the sentiment of the given text message."

user_instruction = """
    Given the following text message: '{text_message}', please evaluate its sentiment by giving a score in the range of -1 to 1, where -1 means negative and 1 means positive.
    Also explain why.
    The output should be in JSON format and follow the following schema:
    --------------
    ```json
    {{
        'score': 0.1,
        'explanation': '...'
    }}
     ```
    """

Let's define the tasks

In [5]:
tasks = []
for index, text_message in enumerate(text_messages):
    task = {
        # The API won't return the input text message, so we need a unique ID for each task
        # This way we can merge the results back with the input text message
        # Instead of generating the ID on the fly, it's recommended to assign a unique ID to each input message at the beginning
        "custom_id": f"text_message_{index}",
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            # This is what you would have in your Chat Completions API call
            "model": "gpt-3.5-turbo",
            "temperature": 0.0,
            "response_format": {
                "type": "json_object"
            },
            "messages": [
                {
                    "role": "system",
                    "content": system_prompt
                },
                {
                    "role": "user",
                    "content": user_instruction.format(text_message=text_message)
                }
            ],
        }
    }

    tasks.append(task)

Now we need to store the results in a file.
Each line is a JSON object.

In [8]:
task_file_name = "text_message_tasks.jsonl"
with open(task_file_name, "w") as f:
    for task in tasks:
        f.write(json.dumps(task) + "\n")

Now we can upload the task file.

In [10]:
batch_file = client.files.create(
    file=open(task_file_name, "rb"),
    purpose="batch"
)

In [11]:
print(batch_file)

FileObject(id='file-9J99vzhJcqoTttVaMiGGAE1L', bytes=3856, created_at=1714186270, filename='text_message_tasks.jsonl', object='file', purpose='batch', status='processed', status_details=None)


We need to write the batch file id down.
It's used to track the status of the batch later.

After this step, you can also view the file at https://platform.openai.com/storage.
Be sure to switch the organization/project associated with your API key.


Now we can create the batch job:

In [ ]:
batch_job = client.batches.create(
    input_file_id=batch_file.id,
    endpoint="/v1/chat/completions",
    completion_window="24h"
)

This step can also be done at https://platform.openai.com/batch using the UI.

After the batch job is done, we can download the results.

In [ ]:
result_file_id = batch_job.output_file_id
result = client.files.content(result_file_id).content

In [ ]:
result_file_name = "openai_text_message_batch_output.jsonl"

with open(result_file_name, 'wb') as file:
    file.write(result)

Now we can read the results.

In [21]:
results = []
with open("openai_text_message_batch_output.jsonl") as f:
    for line in f:
        results.append(json.loads(line))

In [32]:
print(results[0]['response']['body']['choices'][0]['message']['content'])


    {
        "score": 0.9,
        "explanation": "The text message 'The service here is very good!' expresses a positive sentiment with the use of the words 'very good'. This indicates a high level of satisfaction, leading to a positive sentiment score of 0.9."
    }
